# 案例一：Stream 流式输出模式

In [ ]:
from typing import TypedDict
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph
from langgraph.constants import START, END
from langchain_core.messages import AIMessage


# ======================================================
# 1. 定义 State（图的状态结构）
# ------------------------------------------------------
# progress：表示任务进度百分比
# ======================================================
class State(TypedDict):
    progress: int


# ======================================================
# 2. 定义一个简单节点 task_runner
# ------------------------------------------------------
# 每次执行 task_runner：
#   - progress + 25
#   - 返回 {"progress": 新值}
#
# 同时输出一条 AIMessage，用于 messages 模式演示。
# ======================================================
def task_runner(state: State, config: RunnableConfig):
    new_progress = state["progress"] + 25
    msg = AIMessage(content=f"任务执行中，当前进度 = {new_progress}%")
    return {
        "progress": new_progress,
        "messages": [msg]     # 用于 stream_mode="messages"
    }


# ======================================================
# 3. 构造图
# ------------------------------------------------------
# 流程：
#   START → task_runner → END
# ======================================================
builder = StateGraph(State)
builder.add_node("task_runner", task_runner)
builder.add_edge(START, "task_runner")
builder.add_edge("task_runner", END)

graph = builder.compile()


# ======================================================
# 4. 演示四种 stream_mode
# ======================================================

print("\n===================== 1) stream_mode = 'values' =====================")
# 输出每一步的完整 state
for value in graph.stream({"progress": 0}, stream_mode="values"):
    print(value)
# 输出示例： {'progress': 25}


print("\n===================== 2) stream_mode = 'updates' =====================")
# 每次节点更新状态时输出增量（diff）
for update in graph.stream({"progress": 0}, stream_mode="updates"):
    print(update)
# 输出示例：
# {'task_runner': {'progress': 25}}


print("\n===================== 3) stream_mode = 'debug' =====================")
# 输出最详细调试信息，例如：
#   - 执行的节点名
#   - 输入状态
#   - 输出状态
#   - 路由信息
for info in graph.stream({"progress": 0}, stream_mode="debug"):
    print(info)


print("\n===================== 4) stream_mode = 'messages' =====================")
# 用于 LLM 消息流式输出
# task_runner 返回了 AIMessage，因此这里会输出消息
for msg in graph.stream({"progress": 0}, stream_mode="messages"):
    print(msg)


# 可视化图结构
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

# 案例二：Custom 自定义流式输出

In [ ]:
# stream_mode="custom" 的含义
# 自定义流式输出如何工作
# StateGraph 如何流式推送数据
# 这是 LangGraph 自定义流模式（custom stream mode）的最小可复现示例。

from typing import TypedDict
from langgraph.config import get_stream_writer
from langgraph.graph import StateGraph, START


# ======================================================
# 1. 定义状态结构（State）
# ------------------------------------------------------
# request : 输入的请求内容
# response: 节点执行后生成的响应
# ======================================================
class State(TypedDict):
    request: str
    response: str


# ======================================================
# 2. 定义一个节点 processor
# ------------------------------------------------------
# 在 processor 内部使用 get_stream_writer():
#   - 可以向外推送自定义的"流式消息"
#   - 不依赖 LLM，不依赖 AIMessage
#   - 你可以自己定义输出结构，例如 dict、string
#
# writer({"key": "value"}) 会在 stream_mode="custom" 时被捕获
# 并立即被发送给客户端（用户）
# ======================================================
def processor(state: State):
    # 获取一个 writer，该 writer 只在 stream_mode="custom" 时生效
    writer = get_stream_writer()

    # 推送自定义的中间状态消息
    writer({"状态": "开始处理请求"})
    writer({"进度": "50%"})
    writer({"状态": "处理完成"})

    # 节点最终返回新的 State 增量
    return {"response": "请求已处理完毕"}


# ======================================================
# 3. 构建 Graph
# ------------------------------------------------------
# 图结构：
#   START → processor → END
# ======================================================
graph = (
    StateGraph(State)
    .add_node("processor", processor)
    .add_edge(START, "processor")
    .compile()
)


# ======================================================
# 4. 输入数据
# ======================================================
inputs = {"request": "处理订单 #12345"}


# ======================================================
# 5. 使用 stream_mode = "custom"
# ------------------------------------------------------
# 图执行时会有两类流输出：
#
# (1) writer(...) 推送的自定义 streaming 内容
#     → 会被作为 chunk 输出
#
# (2) 节点最终返回的 State 更新结果
#
# 你可以看到整个执行过程中的"实时信息"
# ======================================================
print("\n===== 自定义流式输出 (custom) =====")
for chunk in graph.stream(inputs, stream_mode="custom"):
    print(chunk)

# 可视化图结构
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

# 案例三：LLM Token 流式输出

In [ ]:
from typing import TypedDict
from getpass import getpass
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START


# 1. 获取 API Key
OPENAI_API_KEY = getpass("请输入你的 OpenAI API Key：")


# 2. 初始化 GPT-4o-mini（流式输出）
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY,
    streaming=True
)


# 3. 定义节点
def chat_node(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}


# 4. 创建 LangGraph
builder = StateGraph(MessagesState)
builder.add_node("chat_node", chat_node)
builder.add_edge(START, "chat_node")
graph = builder.compile()


# 5. 输入消息
inputs = {
    "messages": [{"role": "user", "content": "请用一句话介绍 Python 语言"}]
}


# 6. 流式输出大模型 Token（重点）
print("\n=== 流式输出 Token (GPT-4o-mini) ===")
for chunk in graph.stream(inputs, stream_mode="messages"):
    # chunk 是一个元组 (AIMessageChunk, metadata)
    if hasattr(chunk[0], 'content') and chunk[0].content:
        print(chunk[0].content, end="", flush=True)

print()  # 换行

# 可视化图结构
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

# 案例四：Checkpointer 对话记忆

In [ ]:
# ============================================================
# 0. 输入你的 OpenAI API Key（手动输入，不会显示）
# ============================================================
from getpass import getpass
OPENAI_API_KEY = getpass("请输入你的 OpenAI API Key：")


# ============================================================
# 1. 使用 OpenAI 的 GPT-4o-mini 模型
# ------------------------------------------------------------
# ChatOpenAI 是 LangChain 对 OpenAI Chat Completions 的封装
# streaming=True 允许流式输出
# ============================================================
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import InMemorySaver


llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY,
    streaming=True
)


# ============================================================
# 2. 定义一个 LangGraph 节点：调用大模型
# ------------------------------------------------------------
# 输入：MessagesState（包含历史对话 messages）
# 输出：MessagesState（新增模型回复）
# ============================================================
def assistant(state: MessagesState) -> MessagesState:
    response = llm.invoke(state["messages"])
    return {"messages": response}


# ============================================================
# 3. 定义并构建 Graph（状态结构为 MessagesState）
# ------------------------------------------------------------
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_edge(START, "assistant")


# ============================================================
# 4. 初始化 Checkpointer（状态保存器）
# ------------------------------------------------------------
# InMemorySaver 会把每轮对话的状态保存在内存中
# 这样后续的对话可以"记住"之前说过的话
# ============================================================
checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)


# ============================================================
# 5. 配置 config（向 Graph 注入 thread_id）
# ------------------------------------------------------------
# thread_id 是"会话标识符"
# 相同的 thread_id 表示同一个对话，会共享历史记录
# ============================================================
config = {
    "configurable": {
        "thread_id": "chat_session_001"
    }
}


# ============================================================
# 6. 第一次调用：问"什么是机器学习？"
# ------------------------------------------------------------
print("【第一轮对话】")
for chunk in graph.stream(
    {
        "messages": [
            {"role": "user", "content": "什么是机器学习？请简短回答"}
        ]
    },
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()


# ============================================================
# 7. 第二次调用：继续问"它和深度学习有什么区别？"
# ------------------------------------------------------------
# 注意：我们没有重新提供"机器学习"这个上下文
# 但是模型能理解"它"指的是机器学习
# 这就是 Checkpointer 的作用——保持对话上下文
# ============================================================
print("\n【第二轮对话】")
for chunk in graph.stream(
    {
        "messages": [
            {"role": "user", "content": "它和深度学习有什么区别？"}
        ]
    },
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

# 可视化图结构
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# END